In [ ]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from pathlib import Path


from watch_recognition.reports import run_on_image_debug

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# %matplotlib inline

In [ ]:
from watch_recognition.reports import calculate_distances_between_points

In [ ]:
import pandas as pd

In [ ]:
!pwd

In [ ]:

model_path = '../cloud_train/local-training-output/heatmap-regression_098DB017/models/keypoint/efficientnetb0-unet-96-hands/run_1636479122.611962/'
model_path = '../cloud_train/local-training-output/heatmap-regression_3E56883D/models/keypoint/efficientnetb0-unet-96-hands-dl-bce/run_1636825952.284192/'

loaded_model = tf.keras.models.load_model(model_path, compile=False)
image_size=loaded_model.inputs[0].shape[1:3]
loaded_model.summary()

In [ ]:
import cv2

In [ ]:
for path in [
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg"),
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.jpg"),
    Path("../example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(

        path, "rgb", target_size=image_size, interpolation="bicubic",        
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np)

In [ ]:
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
ROOT_DIR = ".."
X, y, filenames = load_keypoints_data_as_kp(
    str(Path("%s/download_data/keypoints/validation" % ROOT_DIR)), autorotate=True,  image_size=image_size,skip_examples_without_all_keypoints=True,
)
X.shape, y.shape

In [ ]:
y_pred = loaded_model.predict(X)

In [ ]:
from watch_recognition.models import *
from watch_recognition.targets_encoding import *
from watch_recognition.datasets import get_watch_keypoints_dataset


In [ ]:
loss = IouLoss2()

In [ ]:
dataset = get_watch_keypoints_dataset(X, y, augment=False, image_size=image_size, shuffle=False,
                                            mask_size=image_size)

In [ ]:
from watch_recognition.datasets import  view_image
view_image(dataset)


In [ ]:
iterator = dataset.as_numpy_iterator()
images = []
losses = []
predictions = []
targets = []
for batch in iterator:
    X_batch, y_batch = batch
    y_pred = loaded_model.predict(X_batch)
    for target, pred in zip(y_batch, y_pred):
        value = loss(target, pred).numpy()
        losses.append(value)
    images.append(X_batch)
    predictions.append(y_pred)
    targets.append(y_batch)
images = np.concatenate(images, axis=0)
predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)
losses = np.array(losses)

In [ ]:
losses

In [ ]:
worst_examples = np.argsort(losses)[::-1][5:10]

In [ ]:
plt.hist(losses, bins='auto')

In [ ]:
losses[worst_examples]

In [ ]:
for idx in np.argsort(losses)[::-1][5:10]:
    print(losses[idx], filenames[idx])
    image = X[idx].astype('uint8')
    colored_mask = (targets[idx] * 255).astype('uint8')
    overlay = cv2.addWeighted(image, 0.35, colored_mask, 0.65, 0)
    plt.imshow(overlay)
    plt.show()
    run_on_image_debug(loaded_model, image)
    diff = np.abs(targets[idx] - predictions[idx])
    plt.imshow(diff)
    plt.show()


In [ ]:
from watch_recognition.datasets import get_watch_keypoints_dataset

In [ ]:
dataset_val = get_watch_keypoints_dataset(
        X,
        y,
        augment=False,
        image_size=X.shape[1:3],
        mask_size=X.shape[1:3],
    ).cache()

In [ ]:
from watch_recognition.datasets import view_image

In [ ]:
view_image(dataset_val)

In [ ]:
pred = loaded_model.predict(dataset_val)

In [ ]:
from watch_recognition.models import IouLoss2
loss = IouLoss2()

In [ ]:
y_target = [batch[1].numpy() for batch in dataset_val]

In [ ]:
X_target = np.concatenate([batch[0].numpy() for batch in dataset_val],axis=0)

In [ ]:
[a.shape for a in y_target]

In [ ]:
targets = np.concatenate(y_target, axis=0)

In [ ]:
pred.shape, targets.shape

In [ ]:
calc_loss = loss(pred, targets)

In [ ]:
loss_values = []
for a, b in zip(pred, targets):
    loss_values.append(loss(a,b).numpy())

In [ ]:
np.argsort(np.array(loss_values))[-3:]

In [ ]:
plt.imshow(X_target[6])

In [ ]:
plt.imshow(pred[6])

In [ ]:
plt.imshow(targets[6])

In [ ]:
loss(targets[6], pred[6]).numpy()

In [ ]:
from watch_recognition.reports import _visualize_high_loss_examples

In [ ]:
%matplotlib inline

In [ ]:
img = _visualize_high_loss_examples(dataset_val, loss, loaded_model)
plt.imshow(img.numpy()[0])
plt.show()

In [ ]:
pred = loaded_model.predict(np.expand_dims(test_image_np, axis=0))
hands = pred[0,:,:,-1]

In [ ]:
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
ROOT_DIR = ".."
X, y, filenames = load_keypoints_data_as_kp(
    str(Path("%s/download_data/keypoints/train" % ROOT_DIR)), autorotate=True,  image_size=image_size,
)
X.shape, y.shape


In [ ]:
predicted = loaded_model.predict(X)

In [ ]:
predicted.shape

In [ ]:
with open('predicted.npz', 'wb') as f:
    np.savez(f, X=X, predicted=predicted, y=y)

In [ ]:
(
    center_distances,
    top_distances,
    hour_distances,
    minute_distances,
) = calculate_distances_between_points(X, predicted, y)
df = pd.DataFrame(
    {
        "center_distances": center_distances,
        "hour_distances": hour_distances,
        "minute_distances": minute_distances,
        "top_distances": top_distances,
        "filename": filenames,
    }
)
df["mean_distances"] = df.mean(axis=1).values
print(df.mean(axis=0))

In [ ]:
from PIL import Image
from watch_recognition.targets_encoding import convert_mask_outputs_to_keypoints

for i, data in df.sort_values("mean_distances", ascending=False).head(5).iterrows():
    scale_factor = X.shape[1] / predicted.shape[1]
    print(f"example {filenames[i]}")
    print(data)
    with Image.open(Path("%s/download_data/keypoints/train" % ROOT_DIR) / filenames[i]) as img:
        print(f"Original resolution: {img.size}")

    center_hat, top_hat, hour_hat, minute_hat  = convert_mask_outputs_to_keypoints(
        predicted[i]
    )
    center_hat, top_hat, hour_hat, minute_hat = [
        np.array(p.as_coordinates_tuple).astype(float)
        for p in (center_hat, top_hat, hour_hat, minute_hat)
    ]
    center_hat = center_hat * scale_factor
    top_hat = top_hat * scale_factor
    hour_hat = hour_hat * scale_factor
    minute_hat = minute_hat * scale_factor
    predicted_points = np.stack(
        [
            center_hat,
            top_hat,
            hour_hat,
            minute_hat,
        ]
    )
   
    rounded_predicted_points = np.round(predicted_points)
    print("targets|predictions")
    print(np.hstack((y[i][:, :2], rounded_predicted_points)))
    markers = ['.', '^', '*', 's']
    names = ['center', 'top', 'hour', 'minute']
    for point, marker, name in zip(predicted_points, markers, names):
        plt.scatter(*point, marker=marker, color='red', label=f"{name}-pred")
    for point, marker, name in zip(y[i][:, :2], markers, names):
        plt.scatter(*point, marker=marker, color='green', label=f"{name}-labeled")

    plt.imshow(X[i])
    ax = plt.gca()
    # https://stackoverflow.com/a/4701285/8814045
    # Shrink current axis's height by 10% on the bottom
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
    plt.show()
    run_on_image_debug(loaded_model, X[i])
    print("#" * 100)

In [ ]:
ROOT_DIR = ".."
X, y, filenames = load_keypoints_data_as_kp(
    str(Path("%s/download_data/keypoints/validation" % ROOT_DIR)), autorotate=True,  image_size=image_size,
)
X.shape, y.shape

In [ ]:
predicted = loaded_model.predict(X)

In [ ]:
(
    center_distances,
    top_distances,
    hour_distances,
    minute_distances,
) = calculate_distances_between_points(X, predicted, y)
df = pd.DataFrame(
    {
        "center_distances": center_distances,
        "hour_distances": hour_distances,
        "minute_distances": minute_distances,
        "top_distances": top_distances,
        "filename": filenames,
    }
)
df["mean_distances"] = df.mean(axis=1).values
print(df.mean(axis=0))